In [1]:
import sys
workdir = '/home/jovyan/workspace'
if not workdir in sys.path:
    sys.path.append(workdir)

In [2]:
import planetengine
system = planetengine.systems.arrhenius.build(res = 64)
initials = {'temperatureField': planetengine.initials.sinusoidal.IC(freq = 1.)}
planetengine.initials.apply(
    initials,
    system,
    )
system.solve()

In [3]:
from planetengine.observer import Observer

In [4]:
class MyObserver(Observer):

    def __init__(
            self,
            _system,
            _initials,
            _outputPath,
            ):
        self.system = _system
        self.initials = _initials
        self.outputPath = _outputPath
        self.scripts = ['test.py']
        self.inputs = {'a': 1}
        self.obsVars = system.varsOfState
        self.collectors = []
        self.figs = []
        super().__init__()

In [8]:
from planetengine import paths

In [10]:
paths.delete_testdir()

In [ ]:
testdir.make_testdir()

In [ ]:
myobs = MyObserver(system, initials, planetengine.testPath)

In [ ]:
myobs.checkpoint()

In [ ]:
system.iterate()

In [ ]:
myobs.update()

In [ ]:
myobs.checkpoint()

In [ ]:
myobs.archive()

In [ ]:
myobs.unarchive()

In [ ]:
import os
myobs.branch(os.path.join(planetengine.testPath, 'subdir'))

In [ ]:
# myvar = planetengine.functions.convert(system.temperatureField)

In [ ]:
# myvars = {
#     'temperature': system.temperatureField,
#     'viscosity': system.viscosityFn
#     }
# myvars = planetengine.functions.convert(myvars)

In [ ]:
# import numpy as np
# with planetengine.tests.TestDir() as outputPath:
#     olddata = np.array(
#         [var.var.data[:] for varName, var in sorted(myvars.items())]
#         )
#     planetengine.disk.varsOnDisk(myvars, outputPath, mode = 'save')
#     for varName, var in sorted(myvars.items()):
#         var.var.data[:] = 0.
#     planetengine.disk.varsOnDisk(myvars, outputPath, mode = 'load')
#     newdata = np.array(
#         [var.var.data[:] for varName, var in sorted(myvars.items())]
#         )
#     assert np.allclose(olddata, newdata)
#     print("Success!")

In [ ]:
# class CustomFrame(_Frame):
#     def __init__(self):

#         system = planetengine.systems.arrhenius.build(res = 16)
#         initials = {'temperatureField': planetengine.initials.sinusoidal.IC(freq = 1.)}
#         planetengine.initials.apply(
#             initials,
#             system,
#             )
#         system.solve()

#         self.system = system
#         self.initials = initials
#         self.outputPath = '/home/jovyan/workspace/data/test'
#         self.instanceID = 'testFrame'
#         self.stamps = {'a': 1}
#         self.step = 0
#         self.onDisk = False
#         self._is_child = False
#         self.inFrames = []
#         self._autoarchive = True
#         checkpointer = checkpoint.Checkpointer(
#             stamps = self.stamps,
#             step = system.step,
#             modeltime = system.modeltime,
#             )
#         mypath = os.path.join(self.outputPath, self.instanceID)
#         def checkpoint(path = None):
#             if path is None:
#                 path = mypath
#             checkpointer.checkpoint(path)
#             self.onDisk = True
#         self.checkpoint = checkpoint
#         def load_checkpoint(step):
#             pass
#         self.load_checkpoint = load_checkpoint

#         super().__init__()